## Define input data

In [1]:
from op_coreutils.duckdb_inmem import init_client
from op_coreutils.partitioned import DataReader
from op_datasets.etl.intermediate.construct import construnct_intermediate_model_batches
from op_datasets.etl.intermediate.udfs import create_duckdb_macros



# Define the input data range.
batches: list[DataReader] = construnct_intermediate_model_batches(
    chains=["op"],
    range_spec="@20241030:+1",
)



# Select input for one date and build the intermediate model inputs.
batch = batches[0]
data_inputs = {}
for dataset in ["blocks", "transactions"]:
    data_inputs[dataset] = batch.duckdb_relation(dataset)


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)

2024-11-06 06:15:10 [info     ] Connecting to OPLABS Clickhouse client...
2024-11-06 06:15:10 [info     ] Loaded 17 items into vault.   
2024-11-06 06:15:10 [info     ] Initialized OPLABS Clickhouse client.
2024-11-06 06:15:10 [info     ] 2024-10-30 data input is ready. chain=op date=2024-10-30
2024-11-06 06:15:10 [info     ] Prepared 1 input batches.      chain=op date=2024-10-30


## Import the function for the model you are working on 

Alos set autoreload so that when you save changes in your logic you do not have to re-import here in the notebook.

In [2]:
from op_datasets.etl.intermediate.models.daily_address_summary import (
    daily_address_summary,
)

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## Run the model

In [ ]:
model_result = daily_address_summary(duckdb_client, data_inputs)

print(sorted(model_result.keys()))

dict_keys(['daily_address_summary_v1'])


## Verify model results

### You can create TEMP tables with the model results and use duckdb:

In [ ]:
for name, relation in model_result.items():
    arrow_relation = relation.to_arrow_table()  # noqa: F841
    duckdb_client.sql(f"CREATE TEMP TABLE {name} AS SELECT * FROM arrow_relation")

In [8]:
duckdb_client.sql("SELECT * FROM daily_address_summary_v1 LIMIT 10")

┌────────────┬─────────┬──────────┬────────────────────────────────────────────┬───────────┬───────────────────┬──────────────┬──────────────────────┬──────────────────┬──────────────────┬───────────────────────┬───────────┬───────────┬───────────────────────┬─────────────────────┬─────────────────────┬──────────────────────┬─────────────────────┬──────────────────┬──────────────────────────┬────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────────┬────────────────────────┬───────────────────────┬───────────────────────┬─────────────────────────────────────┬───────────────────────────┬─────────────────────────────┬─────────────────────────────────┬───────────────────────────────┬───────────────────────┬──────────────────────┬──────────────────────────┬───────────────────────┬───────────────────────────┐
│     dt     │  chain  │ chain_id │                  address                   │ total_txs │ total_txs_suc

### You can also convert the results to dataframes to inspect them in more familiar ways

In [9]:
model_result["daily_address_summary_v1"].pl().head()

dt,chain,chain_id,address,total_txs,total_txs_success,total_blocks,total_blocks_success,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,num_to_addresses,num_to_addresses_success,num_method_ids,total_l2_gas_used,total_l2_gas_used_success,total_l1_gas_used,total_l1_gas_used_success,total_gas_fees,total_gas_fees_success,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_contrib_contrib_gas_fees_blobgas,l1_contrib_gas_fees_l1gas,l2_contrib_gas_fees_basefee,l2_contrib_gas_fees_priorityfee,l2_contrib_gas_fees_legacyfee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei
date,str,i32,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,u32,u32,u32,i64,i64,i64,i64,"decimal[38,0]","decimal[38,0]",f64,f64,"decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]"
2024-10-30,"""op""",10,"""0x724d55da096372f9ff0fbb65ee0d…",5,5,5,5,127348676,127350002,1327,76,80,5,1730296129,1730298781,2652,2,4,4,5,1175446,1175446,12838.0,12838.0,0.0000047585073080740,0.0000047585073080740,0.0000001859917991620,0.0000045725155089120,0.0000028812693172840,0.0000016912461916280,0.0000000004305343690,0.0000001855612647930,0.0000000000000000000,0.0001582310,0.0000003660,0.0001578650,25.2014711050,3.5403502190
2024-10-30,"""op""",10,"""0x508a312cf7334b44b9ff22ac0bac…",7,7,7,7,127339447,127349909,10463,472,478,7,1730277671,1730298595,20924,4,4,4,4,482865,482865,13315.0,13315.0,0.0000023962577638010,0.0000023962577638010,0.0000000484981491570,0.0000023477596146440,0.0000014358548941430,0.0000009119047205010,0.0000000002116491570,0.0000000482865000000,0.0000000000000000000,0.0001004380,0.0000004380,0.0001000000,13.1016862610,1.7011068530
2024-10-30,"""op""",10,"""0x52c635f0c85f2b40f8206a5afca7…",86,86,86,86,127328506,127352446,23941,603,688,86,1730255789,1730303669,47880,7,31,31,29,24750069,24750069,390900.0,390900.0,0.0001764859709471290,0.0001764859709471290,0.0000247601686197590,0.0001517258023273700,0.0001036246791140650,0.0000481011232133050,0.0000000100996197590,0.0000247500690000000,0.0000000000000000000,0.0010004080,0.0000004080,0.0010000000,23.5400962770,4.1817640580
2024-10-30,"""op""",10,"""0x6a5ee6813f537a2b71e9f9588ff8…",160,82,160,82,127324265,127366866,42602,24534,24693,160,1730247307,1730332509,85202,24,1,1,2,45869389,27480957,506238.0,260091.0,0.0001533142556130970,0.0000842350638008920,0.0000556604416522870,0.0000976538139608100,0.0000628049038861720,0.0000348489100746380,0.0000000210264250840,0.0000000000000000000,0.0001532932291880130,0.0012134550,0.0000004580,0.0000000000,13.1676564760,1.9568437670
2024-10-30,"""op""",10,"""0xe3037cf85c662dc23467d3168946…",2,2,2,2,127349882,127349890,9,211,212,2,1730298541,1730298557,16,1,2,2,2,244598,244598,8574.0,8574.0,0.0000034551174445570,0.0000034551174445570,0.0000001106242216520,0.0000033444932229050,0.0000022167587945390,0.0000011277344283660,0.0000000000906945360,0.0000001105335271160,0.0000000000000000000,0.0004522700,0.0000003710,0.0004518990,25.1626721090,4.0786052060
